# KRYLOV Subspace Methods for solving linear systems `Ax=b`

by **Amal MACHTALAY**

### Imports

In [1]:
import numpy as np

## The Steepest Descent algorithm

- Set $x_{0}$ and $r_{0}=b-Ax_{0}$
- **For** $i=1,2,...,N$
   1. $\alpha_{i}=\frac{\left \langle r_{i} , r_{i} \right \rangle}{\left \langle A r_{i},r_{i} \right \rangle}$  
   2. $x_{i+1}=x_{i}+\alpha_{i} r_{i}$
   3. $r_{i+1}=r_{i}-\alpha_{i} A r_{i}$
   4. Stop the iteration when :
$\left \| r_{i} \right \| \leq \varepsilon $.
Where, e.g., $\varepsilon = 1e-8$
- **EndFor**

In [2]:
def Stepseet(A,b,x0,N,e):
  r=np.copy(b)-np.asarray(np.dot(A,x0)).reshape(-1)
  x=[]
  x.append(x0)
  for i in range(N):
    w=np.asarray(np.dot(A,r)).reshape(-1)
    alpha=np.dot(r,r)/np.dot(w,r)
    x0=x0+alpha*r
    r=r-alpha*w
    if np.linalg.norm(r)<=e:
        break
    x.append(x0)
  return x,i

### Test

In [45]:
A=np.matrix('1. 3.;3. -4.')
b=np.array([3.,2.])
x0=np.array([0.,0.])  
x,i=Stepseet(A,b,x0,50,1e-8)
print("The steepest discret algorithm converges in ",i+1," iterations")
#print('x=',x)
print('x=',x[i])

The steepest discret algorithm converges in  19  iterations
x= [1.38461539 0.53846154]


## The Conjugate Gradient algorithm

- Set $x_{0}$ and $p_{0}=r_{0}=b-Ax_{0}$
- **For** $i=1,2,...,N$
   1. $\alpha_{i}=\frac{\left \langle r_{i} , r_{i} \right \rangle}{\left \langle A r_{i},r_{i} \right \rangle}$  
   2. $x_{i+1}=x_{i}+\alpha_{i} r_{i}$
   3. $r_{i+1}=r_{i}-\alpha_{i} A r_{i}$
   4. $\beta_{i+1}=-\frac{\left \| r_{i+1} \right \|^{2}}{\left \| r_{i} \right \|^{2}}$
   5. $p_{i+1}=r_{i+1}+\beta_{i+1} p_{i}$
   6. Stop the iteration when :
$\left \| r_{i} \right \| \leq \varepsilon $.
Where, e.g., $\varepsilon = 1e-8$
- **EndFor**

In [4]:
def Conjugate(A,b,x0,N,e):
  r=np.copy(b)-np.asarray(np.dot(A,x0)).reshape(-1)
  p=np.copy(r)
  x=[]
  x.append(x0)
  for i in range(N):
    w=np.asarray(np.dot(A,r)).reshape(-1)
    alpha=np.dot(r,r)/np.dot(w,r)
    x0=x0+alpha*r
    beta=1/(np.linalg.norm(r)**2)
    r=r-alpha*w
    if np.linalg.norm(r)<=e:
        break
    beta=-beta*(np.linalg.norm(r)**2)
    p=r+beta*p
    x.append(x0)
  return x,i

### Test

In [46]:
A=np.matrix('1. 3.;3. -4.')
b=np.array([3.,2.])
x0=np.array([0.,0.])  
x,i=Conjugate(A,b,x0,50,1e-8)
print("The conjugate gradient algorithm converges in ",i+1," iterations")   
#print('x=',x)
print('x=',x[i])

The conjugate gradient algorithm converges in  19  iterations
x= [1.38461539 0.53846154]


## The Generalized Minimal Residual (GMRES) algorithm

1. Set $x_{0}$ and $r_{0}=b-Ax_{0}$ and $u_{0}=\frac{r_{0}}{\left \| r_{0} \right \|_{2}}$
2. **for** $i=1,2,...,N$
- $w=Au_{i}$ 
- **for** $j=1,..,i$
     - $h_{ji}=\left \langle w , u_{j} \right \rangle$
     - $w=w-h_{ji}u_{j}$
- **End for**
- $h_{i+1,i}=\left \| w \right \|_{2}$
- **If** $h_{i+1,i} \neq 0$ : $u_{i+1}=w/h_{i+1,i}$
- Minimize for $y_{i}$ : $\left \| \beta e_{1} - h y_{i}\right \|$. Where $\beta e_{1}=\left [ \left \| r \right \|_{2} \: 0 \: 0 ... 0 \right ]^{T}$
- $x_{i}=x_{0}+u_{i}y_{i}$
3. Stop the iteration when :
$\left \| r_{i} \right \| \leq \varepsilon $.
Where, e.g., $\varepsilon = 1e-8$
- End

In [6]:
def GMRES(A,b,x0,N,e):
  r=np.copy(b)-np.asarray(np.dot(A,x0)).reshape(-1)
  x=[]
  x.append(x0)
  u=[0]*(N)  # u=[0,0,...,0]
  u[0]=r/np.linalg.norm(r,ord=2)
  h=np.zeros((N+1,N))
  for i in range(N):
    w=np.asarray(np.dot(A,u[i])).reshape(-1)
    if np.linalg.norm(w)<=e:
        break
    for j in range(i):
        h[j,i]=np.dot(u[j],w)
        w=w-h[j,i]*u[j]
    if np.linalg.norm(w)<=e:
        break
    h[i+1,i]=np.linalg.norm(w,ord=2)
    if (h[i+1,i]!=0 and i!=N-1):
      u[i+1]=w/h[i+1,i]
    # calcul of beta*e1
    beta=np.zeros(N+1)
    beta[0]=np.linalg.norm(r,ord=2)
    # Minimize for y
    y=np.linalg.lstsq(h,beta,rcond=-1)[0]
    xN=x0+np.dot(np.asarray(u).transpose(),y)
    x.append(xN)
  return x,i

### Test

In [47]:
A=np.matrix('1. 3.;3. -4.')
b=np.array([3.,2.])
x0=np.array([0.,0.])  
x,i=GMRES(A,b,x0,50,1e-5)
print("The GMRES algorithm converges in ",i+1," iterations")
#print(x)
print(x[i])

The GMRES algorithm converges in  29  iterations
[1.38461375 0.53845889]


## The Bi-Conjugate Gradient Stabilized Method (BICGSTAB) algorithm

1. Set $x_{0}$, $r_{0}=b-Ax_{0}$, $p_{0}=r_{0}$ and $\overline{r}_{0}$ such that $r_{0}.\overline{r}_{0} \neq 0$
2. **for** $i=1,2,...,N$
- $\alpha_{i}=\frac{\left \langle r_{i} , \overline{r}_{0} \right \rangle}{\left \langle A p_{i},\overline{r}_{0} \right \rangle}$ 
- $s_{i}=r_{i}-\alpha_{i}A p_{i}$
- **If** $\left \| s_{i} \right \| \leq \varepsilon $ : 
  - $x_{i+1}=x_{i}+\alpha_{i}p_{i}$
  - **Break**
- **End If**
- $\omega_{i}=\frac{\left \langle As_{i} , s_{i} \right \rangle}{\left \langle A s_{i},As_{i} \right \rangle}$ 
- $x_{i+1}=x_{i}+\alpha_{i}p_{i}+\omega_{i}s_{i}$
- $r_{i+1}=s_{i}-\omega_{i}As_{i}$
- **If** $\left \| r_{i+1} \right \| \leq \varepsilon $ : 
  - **Break**
- **End If**
- $\beta_{i}=\frac{\alpha_{i}}{\omega_{i}} \frac{\left \langle r_{i+1} , \overline{r}_{0} \right \rangle}{\left \langle r_{i},\overline{r}_{0} \right \rangle}$
- $p_{i+1}=r_{i+1}+\beta_{i}(p_{i}-\omega_{i}Ap_{i})$
- **End for**
3. $x=x_{i+1}$

In [8]:
def BICGSTAB(A,b,x0,N,e):
  r=np.copy(b)-np.asarray(np.dot(A,x0)).reshape(-1)
  rbar=np.copy(b)
  p=np.copy(r)
  x=[]
  x.append(x0)
  for i in range(N):
    Ap=np.asarray(np.dot(A,p)).reshape(-1)
    alpha=np.dot(r,rbar)/np.dot(Ap,rbar)
    s=r-alpha*Ap
    if np.linalg.norm(s)<=e:
        x0=x0+alpha*p
        break
    As=np.asarray(np.dot(A,s)).reshape(-1)
    Omega=np.dot(As,s)/np.dot(As,As)
    x0=x0+alpha*p+Omega*s
    r=s-Omega*As
    beta=(alpha/Omega)/np.dot(r,rbar)
    if np.linalg.norm(r)<=e:
        break
    beta=beta*np.dot(r,rbar)
    p=r+beta*(p-Omega*Ap) 
    x.append(x0)
  return x,i

### Test

In [48]:
A=np.matrix('1. 3.;3. -4.')
b=np.array([3.,2.])
x0=np.array([0.,0.])  
x,i=BICGSTAB(A,b,x0,50,1e-8)
print("The BICGSTAB algorithm converges in ",i+1," iterations")   
#print('x=',x)
print('x=',x[i])

The BICGSTAB algorithm converges in  6  iterations
x= [1.38461539 0.53846154]


## The Symmetric Lanczos Algorithm

1. Set $x_{0}$, $r_{0}=b-Ax_{0}$ and $v_{0}=\frac{r_{0}}{\beta_{0}}$, $\beta_{0}=\left \| r_{0} \right \|_{2}$
2. **for** $i=1,2,...,m$
- $w=Au_{i}-\beta_{i}v_{i-1}$ 
- $\alpha_{i}=\left \langle w , v_{i} \right \rangle$
- $w=w-\alpha_{i}v_{i}$
- $\beta_{i+1}=\left \| w \right \|_{2}$
- **If** $\beta_{i+1} \neq 0$ : $v_{i+1}=w/\beta_{i+1}$
- **End for** 
- $y_{m}=\beta T_{m}^{-1}e_{1}$ and $x_{m}=x_{0}+V_{m}y_{m}$.

In [13]:
def Symmetric_Lanczos(A,b,x0,m):
  r=b-np.dot(A,x0)
  betha=np.linalg.norm(r)
  v=r/betha
  V=np.zeros([m,b.size])
  V[0,:]=v
  a=np.zeros(m)
  bi=np.zeros(m)
  e1=np.zeros(m)
  e1[0]=1
  for j in range(m):
    w=A.dot(V[j,:])-bi[j]*V[j-1,:]
    a[j]=w.dot(V[j,:])
    w=w-a[j]*V[j,:]
    B=np.linalg.norm(w,ord=2)
    if (B==0) or (j==m-1):
        break
    bi[j+1]=B
    V[j+1,:]=w/B
  Tm=np.diag(a)+np.diag(bi[1:],1)+np.diag(bi[1:],-1)
  ym=betha*np.linalg.inv(Tm).dot(e1)
  xm=x0+V.transpose().dot(ym)
  return xm,j

In [49]:
A=np.matrix('1. 3.;3. -4.')
b=np.array([3.,2.])
x0=np.array([0.,0.])  
x,i=Symmetric_Lanczos(A,b,x0,2)
print("The Symmetric Lanczos algorithm converges in ",i+1," iterations")   
print('x=',x)

The Symmetric Lanczos algorithm converges in  2  iterations
x= [1.38461538 0.53846154]
